In [59]:
import db_util as db
import json
import pymysql
import random
import pandas as pd

In [56]:
with open('data/mysql.json', 'r') as file: 
    config_str = file.read() 
    config = json.loads(config_str)

In [47]:
db.dl_db('products')
db.dl_db('sales')

In [48]:
db.mk_pdb()
db.mk_sdb()

카테고리  
자동차, 인테리어, 과자
#product db 만들기  pid, 제품명, 가격, 카테고리, 원가

In [49]:
cat = ['자동차관련', '가전제품 및 인테리어', '과자']
car = ['와이퍼', '타이어', '핸들', '휠', '시트', '에어컨', '뺵미러', '엔진', '엔진오일', '내비게이션']
inte = ['김치냉장고', '냉장고', '컴퓨터', '식기세척기', '책상', '의자', '티비', '전자레인지', '오븐', '인덕션']
snack = ['맛동산', '바나나킥', '빈츠', '오감자', '초코파이', '몽쉘', '참치크래커', '고향만두', '닭강정', '오레오']

In [50]:
sql = """
INSERT INTO products(pname , pprice , pcategory , pcost) 
VALUES(%s, %s, %s, %s)
"""
conn = pymysql.connect(**config)
cur = conn.cursor()

In [51]:
import random
random.seed(2021)
for i, item_list in enumerate([car, inte, snack]):
    for item in item_list:
        #가격
        price = random.randint(100, 500)*100
        cost = int(price * (random.randint(55,99)/100))
        cur.execute(sql, (item, price, cat[i], cost))
        #db.i_pdb([item, price,cat[i], cost]) 모듈쓰면 시간 너무 길어진다
        #원가
conn.commit()

달력 랜덤
1~ 12 랜덤, 1~ 28 랜덤
date, scompany , spid, sunit

회사는 5개 sid 는 랜덤하계 한번씩 나오도록 가능
판 횟수도 랜던 

In [52]:
sql = """
    INSERT INTO sales(sdate, scompany , spid, sunit) 
    VALUES(%s, %s, %s, %s)
"""
conn = pymysql.connect(**config)
cur = conn.cursor()

In [53]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
random.seed(2021)
companys = ['송샘', 'RG', 'DOROKO', 'glo', '다이서']
for m, day in calendar.items():
    for i in range(10):
        date = f'2020-{m}-{random.randint(1,day)}'
        company = companys[random.randint(0,4)]
        spid = random.randint(1,30)
        sunit = random.randint(10,20)
        cur.execute(sql, (date, company, spid, sunit))
        #db.i_sdb([date, company, spid, sunit]) 모듈쓰면 시간 너무 길어진다

conn.commit()

In [57]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""
conn = pymysql.connect(**config)
cur = conn.cursor()
cur.execute(sql)

0

In [60]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,103,2020-01-02,glo,16,의자,가전제품 및 인테리어,42500,19,35275,807500,137275
1,104,2020-01-03,DOROKO,9,엔진오일,자동차관련,18200,14,11830,254800,89180
2,106,2020-01-06,송샘,24,오감자,과자,15900,11,15264,174900,6996
3,105,2020-01-16,송샘,30,오레오,과자,48700,12,33603,584400,181164
4,110,2020-01-17,DOROKO,22,바나나킥,과자,22400,15,18816,336000,53760


In [61]:
del df[3]
del df[0]

In [62]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-02,glo,의자,가전제품 및 인테리어,42500,19,35275,807500,137275
1,2020-01-03,DOROKO,엔진오일,자동차관련,18200,14,11830,254800,89180
2,2020-01-06,송샘,오감자,과자,15900,11,15264,174900,6996
3,2020-01-16,송샘,오레오,과자,48700,12,33603,584400,181164
4,2020-01-17,DOROKO,바나나킥,과자,22400,15,18816,336000,53760


In [63]:
df.to_csv('data/sales_book.csv', index=False)

In [64]:
conn.close()
cur.close()